## Caso de estudio 1: Control del ángulo del motor en variables de estado

Dadas las ecuaciones del motor de corriente continua con las mediciones experimentales detalladas en la Fig. 1, se sabe que las ecuaciones son:

1. $$ \frac{di_a}{dt} = -\frac{R_A}{L_{AA}}i_a - \frac{K_m}{L_{AA}}\omega_r + \frac{1}{L_{AA}}v_a $$
2. $$ \frac{d\omega_r}{dt} = \frac{K_i}{J}i_a - \frac{B_m}{J}\omega_r - \frac{1}{J}T_L $$
3. $$ \frac{d\theta_t}{dt} = \omega_r $$

---

**Ítem [1]**  
Empleando los parámetros hallados en el Trabajo Práctico Nº1 para el Motor CC, implementar un sistema en variables de estado que controle el ángulo del motor, para consignas de π/2 y –π/2 cambiando cada 5 segundos y que el $ T_L $ es el descripto en la planilla de datos, comparando el desempeño con el obtenido con el PID digital del TP Nº1. Hallar el valor de integración Euler adecuado.

**Objetivo:** acelerar la dinámica del controlador verificando el resultado con las curvas del archivo `.xls` adjunto.

- **a.** Evitando que la tensión supere los 5 Volts en valor absoluto, especificar el tiempo de muestreo necesario para que el controlador cumpla el objetivo.  
- **b.** Asumiendo que no puede medirse directamente la corriente, **pero sí la velocidad y el ángulo**, proponer un controlador que logre el objetivo.  
- **c.** Determinar el efecto de la no linealidad en la acción de control, descripta en la Fig. 2, y verificar cuál es el máximo valor admisible de esa no linealidad.

![Caso 1 - imagen](extras/case1.png)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import place_poles

 ### 1. Modelo del Motor CC
 La representación en espacio de estados es:
 $$
 \dot{x} = A x + B u, \quad y = C x
 $$
 donde:
 $$
 x = \begin{bmatrix} i_a \\ \omega_r \\ \theta_r \end{bmatrix}, \quad
 u = \begin{bmatrix} v_a \\ T_L \end{bmatrix}, \quad
 y = \theta_r
 $$
 $$
 A = \begin{bmatrix}
 -\frac{R_A}{L_{AA}} & -\frac{K_m}{L_{AA}} & 0 \\
 \frac{K_i}{J} & -\frac{B_m}{J} & 0 \\
 0 & 1 & 0
 \end{bmatrix}, \quad
 B = \begin{bmatrix}
 \frac{1}{L_{AA}} & 0 \\
 0 & -\frac{1}{J} \\
 0 & 0
 \end{bmatrix}, \quad
 C = \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}
 $$

In [8]:
# ------------------- Parámetros del Sistema -------------------
# Parámetros del motor (identificados experimentalmente)
Ra = 2.2781228953606902
Laa = 0.005187184919244553
Ki = 0.2618711775870197
Jm = 0.002848787974411428
Bm = 0.0014279727330389095
Km = 0.2499435593696499


# Parámetros del motor [originales]
# Ra = 2.27        # Resistencia de armadura [Ohm]
# Laa = 0.0047     # Inductancia de armadura [H]
# Ki = 0.25        # Constante de torque [N⋅m/A]
# Km = 0.25        # Constante de fuerza contraelectromotriz [V⋅s/rad]
# Bm = 0.00131     # Coeficiente de fricción viscosa [N⋅m⋅s/rad]
# Jm = 0.00233     # Momento de inercia [kg⋅m²]


# Matrices del sistema en espacio de estados
A = np.array([
    [-Ra/Laa,   -Km/Laa,  0],
    [Ki/Jm,     -Bm/Jm,   0],
    [0,         1,        0]
])
B_va = np.array([[1/Laa], [0], [0]])
B_torque = np.array([[0], [-1/Jm], [0]])
B = np.hstack((B_va, B_torque))
C = np.array([[0, 0, 1]])



## Integrador para la realimentación de estados
Para esto, antes agregamos el integrador (para que el error converja a cero)



El sistema ampliado se convierte en:

$$
\begin{bmatrix}
\dot{x}(t) \\
\dot{\xi}(t)
\end{bmatrix}
=
\begin{bmatrix}
A & 0 \\
-C & 0
\end{bmatrix}
\begin{bmatrix}
x(t) \\
\xi(t)
\end{bmatrix}
+
\begin{bmatrix}
B \\
0
\end{bmatrix}
u(t)

$$


Donde:

$$

A_a = \begin{bmatrix} A & 0 \\ -C & 0 \end{bmatrix}, \quad

x_a = \begin{bmatrix} x \\ \xi \end{bmatrix}

B_a = \begin{bmatrix} B \\ 0 \end{bmatrix}, \quad

E_a = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \quad
$$


In [ ]:
# Primero agregamos el integrador y hacemos la matriz aumentada
A_aug = np.vstack((np.hstack((A, np.zeros((3,1)))), 
                   np.hstack((-C[0,:], 0)) ))

B_aug = np.vstack((B_va, 0))
C_aug = np.hstack((C, np.array([[0]])))

# Construiremos el observador de Luenberger

Tenemos 2 caminos para diseñar el observador:

1. Transformamos la matriz en la forma canonica observable, luego despejar para calcular la ganancia Ko (L)
2. Aprovechar el concepto de dualidad para hallar Ko(L)

En este caso iremos por la segunda opción. Ya que nos permite transformar el problema de diseño del observador en un problema de control dual.

Para el sistema original:
$$
\begin{align}
\dot{x} &= Ax + Bu \\
y &= Cx
\end{align}
$$

El sistema dual es:
$$
\begin{align}
\dot{x} &= A^T x + C^T u \\
y &= B^T x
\end{align}
$$

Donde:
 - $A_{dual} = A^T$ (transpuesta de $A$)
 - $B_{dual} = C^T$ (transpuesta de $C$)
 - $C_{dual} = B^T$ (transpuesta de $B$)

La ventaja es que si el sistema original es observable, el sistema dual es controlable.
Esto nos permite usar las técnicas de control (como ubicación de polos) para encontrar la ganancia del observador L (Ko)

La ganancia del observador L será la transpuesta de la ganancia K del controlador del sistema dual:
$L = K^T$


In [14]:
# Verificamos que el sistema original sea observable
O = np.hstack((B, A@B, (A)**2 @B))
rank_O = np.linalg.matrix_rank(O)

# Si el rango es igual a la dimensión del sistema, el sistema es observable
if rank_O == A.shape[0]:
    print("El sistema es observable")
else:
    print("El sistema no es observable")



# Sistema dual
A_dual = A.T
B_dual = C.T
C_dual = B_va.T

# Chequeo de dimensiones (no es necesario, lo dejo para debugging)
print("Dimensiones:")
print(f"A_dual: {A_dual.shape}")
print(f"B_dual: {B_dual.shape}")
print(f"C_dual: {C_dual.shape}")
 


El sistema es observable
Dimensiones:
A_dual: (3, 3)
B_dual: (3, 1)
C_dual: (1, 3)


Elegimos los polos del observador de manera que el sistema sea estable y que el error converja a cero en el tiempo deseado. Evitamos que el observador sea demasiado rapido, para que no se vuelva inestable.


In [24]:
# Calculamos la dinámica más rápida del sistema original
max_pole = np.max(np.abs(np.linalg.eigvals(A)))

print(max_pole)
# Calculamos los polos del observador de manera que sea 4 veces más rapido que el sistema
# polos_observador = polos_sistema * 2 
# print(polos_observador)
 

 # Para este caso, consideramos que el tiempo de muestreo es 0.001 s
t_muestreo = 0.001

print("En teoría, los polos del observador deberían ser mayores a ", 10*max_pole)

# Se hace caso a la sugerencia de la catedra, y se eligen los siguientes polos 
# para el observador

polos_observador = np.array([-330, -200, -220]) 



428.84223830964004
En teoría, los polos del observador deberían ser mayores a  4288.4223830964
